In [1]:
import torch
import data
from helpers import Config

In [2]:
SEED = 42
DATA = '/floyd/home/sein/'
CUDA = False
LOG_INTERVAL = 100
CONFIG_NAME = 'text_generation_base'
device = torch.device("cuda" if CUDA else "cpu")
args = Config(CONFIG_NAME)
assert args.temperature >= 1e-3, "temperature has to be greater or equal 1e-3"

In [3]:
corpus = data.Corpus(DATA)
ntokens = len(corpus.dictionary)

In [4]:
with open(args.checkpoint, 'rb') as f:
    model = torch.load(f, map_location=device)
model.eval()

WD_LSTM(
  (variational_dropout): VariationalDropout()
  (encoder): Embedding(18818, 400)
  (rnns): ModuleList(
    (0): WeightDrop(
      (module): LSTM(400, 1150)
    )
    (1): WeightDrop(
      (module): LSTM(1150, 1150)
    )
    (2): WeightDrop(
      (module): LSTM(1150, 400)
    )
  )
  (decoder): Linear(in_features=400, out_features=18818, bias=True)
)

In [5]:
hidden = model.init_hidden(1)
cue_words = ["george", ":", "will"]
cue = torch.tensor([[corpus.dictionary.word2idx[cue_words[0]]]]).to(device)

In [6]:
with open(args.outf, 'w') as outf:
    #outf.write(" ".join(cue_words))
    with torch.no_grad():  # no tracking history
        for i in range(args.words):
            output, hidden, _, _ = model(cue, hidden)
            word_weights = output.squeeze().div(args.temperature).exp().cpu()
            word_idx = torch.multinomial(word_weights, 1)[0]
            cue.fill_(word_idx)
            word = corpus.dictionary.idx2word[word_idx]
            outf.write(word + ('\n' if i % 20 == 19 else ' '))
            if i % LOG_INTERVAL == 0:
                print('| Generated {}/{} words'.format(i, args.words))

| Generated 0/200 words
| Generated 100/200 words
